In [1]:
# Load Data
import glob 
import pandas as pd

def Carga_All_Files( ):
    regexp='../data/covi*'
    df = pd.DataFrame()
    # Iterate trough LIST DIR and 
    for my_file in glob.glob(regexp):
        this_df = pd.read_csv(my_file)
        this_df['Fecha'] = my_file
        df = pd.concat([df,this_df])
    return df  

df = Carga_All_Files( )

df.tail()


ID        CCAA  Casos     IA  UCI  Fallecidos  Hospitalizados  Nuevos  \
13  942      Madrid   1388  20.77  135          56             NaN     NaN   
14  952      Murcia     26   1.74    1           0             NaN     NaN   
15  943     Navarra     73  11.16    3           0             NaN     NaN   
16  950  País Vasco    346  15.67   18          11             NaN     NaN   
17  949    La Rioja    205  64.71    1           2             NaN     NaN   

    Curados                 Fecha  
13      NaN  ../data/covi1203.csv  
14      NaN  ../data/covi1203.csv  
15      NaN  ../data/covi1203.csv  
16      NaN  ../data/covi1203.csv  
17      NaN  ../data/covi1203.csv

In [2]:
def Get_Comunidades_List( ):
    return Carga_All_Files( )['CCAA'].unique()

In [3]:
#Get_Comunidades_List()

In [4]:
def Preprocesado():
    df = Carga_All_Files( )
    # Formateamos la fecha
    df['Fecha'].replace({
        '../data/covi': '2020-',
        '.csv'        : ''}, inplace=True, regex=True)
    df['Fecha'] =  pd.to_datetime(df['Fecha'], format='%Y-%d%m')
    # 
    
    return df.sort_values(by='Fecha')

df = Preprocesado()


In [5]:
import numpy as np

def Enrich_Columns(comunidad):
    del comunidad['ID']
    del comunidad['IA']
    del comunidad['Nuevos']

    if 'Fecha' in comunidad.columns :
        comunidad.set_index('Fecha', inplace=True) 

    # Datos de fallecimientos diarios, en totales y tanto por uno.
    comunidad['Fallecidos hoy absoluto'] = comunidad['Fallecidos'] - comunidad['Fallecidos'].shift(1)
    comunidad['Fallecidos hoy porcentaje'] = comunidad['Fallecidos hoy absoluto']  / comunidad['Fallecidos'] 
    comunidad['Fallecidos hoy variacion respecto ayer'] = comunidad['Fallecidos hoy absoluto'] - comunidad['Fallecidos hoy absoluto'].shift(1)

    # Datos de Casos diarios,  en totales y tanto por uno.
    comunidad['Casos hoy absoluto'] = comunidad['Casos'] - comunidad['Casos'].shift(1)
    comunidad['Casos hoy porcentaje'] = comunidad['Casos hoy absoluto']  / comunidad['Casos'] 
    comunidad['Casos hoy variacion respecto ayer'] = comunidad['Casos hoy absoluto'] - comunidad['Casos hoy absoluto'].shift(1)

    # Convertimos a entero, para quitar decimales
    CONVERT_INT_COLUMNS = ['Fallecidos hoy absoluto', 
                           'Fallecidos hoy variacion respecto ayer',
                           'Casos hoy variacion respecto ayer',
                           'Casos hoy absoluto',
                           'Hospitalizados',
                           'Curados']
    for column in CONVERT_INT_COLUMNS :
        comunidad[column] = comunidad[column].fillna(0)
        comunidad[column] = comunidad[column].astype(np.int64)
        
    comunidad['Proporcion Curados / Casos'] = comunidad['Curados'] / comunidad['Casos'] 

    # ordenamos las filas y columnas
    columnsTitles = ['CCAA', 
                     'Casos'     , 'Casos hoy absoluto'     , 'Casos hoy variacion respecto ayer', 'Casos hoy porcentaje'      ,
                     'Fallecidos', 'Fallecidos hoy absoluto', 'Fallecidos hoy variacion respecto ayer', 'Fallecidos hoy porcentaje' ,
                     'Curados',
                     'Proporcion Curados / Casos',
                     'UCI',  
                     'Hospitalizados']
    comunidad = comunidad.reindex(columns=columnsTitles)
    comunidad = comunidad.sort_values(by=['Fecha'], ascending=False)
    comunidad = comunidad.rename(columns = {'CCAA':'Lugar'})

    return comunidad

def Get_Comunidad(nombre_comunidad):
    # Trabajamos solo con una comunidad
    df = Preprocesado()
    df = df[(df['CCAA'] == nombre_comunidad)].sort_values(by='Fecha')
    df = Enrich_Columns(df)
    return df

def Get_Nacion():
    df = Preprocesado()
    df = df.sort_values(by='Fecha')
    df = df.groupby(['Fecha']).sum()
    df['CCAA'] = 'España'
    df = Enrich_Columns(df)
    return df


In [6]:
# Just for debug purposes
def Debug_Get_Comunidad():
    comunidad = Get_Comunidad('Madrid')
    return comunidad

Debug_Get_Comunidad()



Lugar  Casos  Casos hoy absoluto  \
Fecha                                           
2020-03-26  Madrid  17166                2569   
2020-03-25  Madrid  14597                2245   
2020-03-24  Madrid  12352                1777   
2020-03-23  Madrid  10575                 873   
2020-03-22  Madrid   9702                 781   
2020-03-21  Madrid   8921                1756   
2020-03-20  Madrid   7165                 388   
2020-03-19  Madrid   6777                1140   
2020-03-18  Madrid   5637                 766   
2020-03-17  Madrid   4871                 706   
2020-03-16  Madrid   4165                2175   
2020-03-13  Madrid   1990                 602   
2020-03-12  Madrid   1388                 364   
2020-03-11  Madrid   1024                 242   
2020-03-10  Madrid    782                 313   
2020-03-09  Madrid    469                 332   
2020-03-06  Madrid    137                  47   
2020-03-05  Madrid     90                   0   

            Casos hoy variacion respecto ayer  Casos hoy porcentaje  \
Fecha                                                                 
2020-03-26                                324              0.149656   
2020-03-25                                468              0.153799   
2020-03-24                                904              0.143863   
2020-03-23                                 92              0.082553   
2020-03-22                               -975              0.080499   
2020-03-21                               1368              0.196839   
2020-03-20                               -752              0.054152   
2020-03-19                                374              0.168216   
2020-03-18                                 60              0.135888   
2020-03-17                              -1469              0.144939   
2020-03-16                               1573              0.522209   
2020-03-13                                238              0.302513   
2020-03-12                                122              0.262248   
2020-03-11                                -71              0.236328   
2020-03-10                                -19              0.400256   
2020-03-09                                285              0.707889   
2020-03-06                                  0              0.343066   
2020-03-05                                  0                   NaN   

            Fallecidos  Fallecidos hoy absoluto  \
Fecha                                             
2020-03-26        2090                      265   
2020-03-25        1825                      290   
2020-03-24        1535                      272   
2020-03-23        1263                      242   
2020-03-22        1021                      217   
2020-03-21         804                      176   
2020-03-20         628                      130   
2020-03-19         498                      108   
2020-03-18         390                       35   
2020-03-17         355                      142   
2020-03-16         213                      132   
2020-03-13          81                       25   
2020-03-12          56                       25   
2020-03-11          31                       10   
2020-03-10          21                       13   
2020-03-09           8                        6   
2020-03-06           2                        1   
2020-03-05           1                        0   

            Fallecidos hoy variacion respecto ayer  Fallecidos hoy porcentaje  \
Fecha                                                                           
2020-03-26                                     -25                   0.126794   
2020-03-25                                      18                   0.158904   
2020-03-24                                      30                   0.177199   
2020-03-23                                      25                   0.191607   
2020-03-22                                      41                   0.212537   
2020-03-21             

In [7]:
# Just for debug purposes
def Debug_Get_Nacion():
    return Get_Nacion()

Debug_Get_Nacion()


Lugar  Casos  Casos hoy absoluto  \
Fecha                                           
2020-03-26  España  56188                8578   
2020-03-25  España  47610                7937   
2020-03-24  España  39673                6584   
2020-03-23  España  33089                4517   
2020-03-22  España  28572                3646   
2020-03-21  España  24926                4946   
2020-03-20  España  19980                2833   
2020-03-19  España  17147                3431   
2020-03-18  España  13716                2538   
2020-03-17  España  11178                1987   
2020-03-16  España   9191                4982   
2020-03-13  España   4209                1259   
2020-03-12  España   2950                 822   
2020-03-11  España   2128                 506   
2020-03-10  España   1622                 623   
2020-03-09  España    999                 663   
2020-03-06  España    336                  99   
2020-03-05  España    237                   0   

            Casos hoy variacion respecto ayer  Casos hoy porcentaje  \
Fecha                                                                 
2020-03-26                                641              0.152666   
2020-03-25                               1353              0.166709   
2020-03-24                               2067              0.165957   
2020-03-23                                871              0.136511   
2020-03-22                              -1300              0.127607   
2020-03-21                               2113              0.198427   
2020-03-20                               -598              0.141792   
2020-03-19                                893              0.200093   
2020-03-18                                551              0.185039   
2020-03-17                              -2995              0.177760   
2020-03-16                               3723              0.542052   
2020-03-13                                437              0.299121   
2020-03-12                                316              0.278644   
2020-03-11                               -117              0.237782   
2020-03-10                                -40              0.384094   
2020-03-09                                564              0.663664   
2020-03-06                                  0              0.294643   
2020-03-05                                  0                   NaN   

            Fallecidos  Fallecidos hoy absoluto  \
Fecha                                             
2020-03-26        4089                      655   
2020-03-25        3434                      738   
2020-03-24        2696                      514   
2020-03-23        2182                      462   
2020-03-22        1720                      394   
2020-03-21        1326                      324   
2020-03-20        1002                      235   
2020-03-19         767                      169   
2020-03-18         598                      107   
2020-03-17         491                      182   
2020-03-16         309                      191   
2020-03-13         118                       34   
2020-03-12          84                       37   
2020-03-11          47                       12   
2020-03-10          35                       19   
2020-03-09          16                       11   
2020-03-06           5                        2   
2020-03-05           3                        0   

            Fallecidos hoy variacion respecto ayer  Fallecidos hoy porcentaje  \
Fecha                                                                           
2020-03-26                                     -83                   0.160186   
2020-03-25                                     224                   0.214910   
2020-03-24                                      52                   0.190653   
2020-03-23                                      68                   0.211732   
2020-03-22                                      70                   0.229070   
2020-03-21             